In [26]:
import nltk
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
stopWords = set(stopwords.words('english'))

In [27]:
df = pd.read_csv('Emotion_final.csv')

In [28]:
labels = sorted(list(set(df.Emotion)))
for i in range(len(labels)):
    df.Emotion[df.Emotion == labels[i]] = i

In [29]:
labels_map = {i: labels[i] for i in range(len(labels))}
labels_map

{0: 'anger', 1: 'fear', 2: 'happy', 3: 'love', 4: 'sadness', 5: 'surprise'}

In [30]:
def tokenizer(text):
    tokenized_list = []

    # t = ['feel', 'feeling']

    lemantaizer = WordNetLemmatizer()
    t = nltk.word_tokenize(text)
    for word in nltk.word_tokenize(text):
        if word.isalpha():
            word=word.lower()
            word=lemantaizer.lemmatize(word)
            if word not in stopWords or word not in t:    
                tokenized_list.append(word)
    return ' '.join(tokenized_list)

In [31]:
for i in range(len(df)):
    df.Text[i] = tokenizer(df.Text[i])

# for i in range(len(df_test)):
#     df_test.Message_body[i] = tokenizer(df.Message_body[i])

In [32]:
X = df['Text']
y = df['Emotion']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)


In [33]:
vectorizer = TfidfVectorizer(stop_words='english')

tf_idf = vectorizer.fit(X_train)

X_train = tf_idf.transform(X_train)
X_test = tf_idf.transform(X_test)

In [34]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
forest = RandomForestClassifier(n_estimators=300, max_depth=300).fit(X_train, y_train.astype(np.int64))

# pred = forest.predict(X_train)
# print(classification_report(y_train.astype(np.int64), pred))
pred = forest.predict(X_test)
print(classification_report(y_test.astype(np.int64), pred))

              precision    recall  f1-score   support

           0       0.92      0.83      0.87       993
           1       0.85      0.82      0.84       871
           2       0.84      0.94      0.89      2279
           3       0.86      0.71      0.78       550
           4       0.90      0.89      0.90      2110
           5       0.79      0.73      0.76       279

    accuracy                           0.87      7082
   macro avg       0.86      0.82      0.84      7082
weighted avg       0.87      0.87      0.87      7082



# LDA

In [35]:
idfs = tf_idf.idf_